# 🎓 **Inteligencia Artificial Aplicada**

## 🤖 **Operaciones de aprendizaje automático (Gpo 10)**

### 🏛️ Tecnológico de Monterrey

#### 👨‍🏫 **Profesor titular :** Dr. Gerardo Rodríguez Hernández
#### 👩‍🏫 **Profesor titular :** Maestro Ricardo Valdez Hernández
#### 👩‍🏫 **Profesor tutor :** Jorge Gonzales Zapata

### 📊 **Fase 1 Proyecto MLOps**

#### 📅 **Octubre de 2025**

### 👥 Equipo 43

* 🧑‍💻 **A01795645 :** Alberto Campos Hernández
* 🧑‍💻 **A01016093 :** Oscar Enrique García García
* 🧑‍💻 **A01795922 :** Jessica Giovana García Gómez
* 🧑‍💻 **A01795897 :** Esteban Sebastián Guerra Espinoza
* 🧑‍💻 **A00820345 :** Rafael Sánchez Marmolejo

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import skew, kurtosis
from __future__ import annotations
from dataclasses import dataclass
from typing import Tuple, Optional, List
from pathlib import Path
# Configuración global de paralelismo para todos los modelos
N_JOBS = 2

In [ ]:
# ==========================
# CARGA DE ARCHIVOS
# ==========================
@dataclass
class CargaArchivos:
    """Carga datasets crudos desde una carpeta.

    Parámetros
    ----------
    carpeta_raw: str | Path
        Ruta a la carpeta que contiene los CSVs crudos.
    nombre_modificado: str
        Nombre del CSV "modificado". power_tetouan_city_modified.csv
    """

    carpeta_raw: Path
    nombre_modificado: str

    def __post_init__(self) -> None:
        self.carpeta_raw = Path(self.carpeta_raw)
        self.carpeta_raw.mkdir(parents=True, exist_ok=True)

    def leer(self) -> pd.DataFrame:
        na_vals = ["nan", "NAN", "NaT", ""]
        df_modificado = pd.read_csv(
            self.carpeta_raw / self.nombre_modificado,
            na_values=na_vals,
            keep_default_na=True,
        )
        return df_modificado


# ==========================
# PREPROCESAMIENTO
# ==========================
@dataclass
class Preprocesamiento:
    """Transforma el dataset modificado en un dataset listo para modelar.
    Pasos realizados:
    - Elimina columna "mixed_type_col" si existe.
    - Limpia y convierte DateTime con distintos formatos.
    - Imputa DateTime faltante con vecino a ±10 min o punto medio.
    - Imputa numéricos con mediana por columna.
    - Maneja outliers mediante IQR + mediana rodante (ventana configurable).
    - Crea variables de tiempo y elimina DateTime si se solicita.
    """

    ventana_mediana: int = 25
    eliminar_datetime: bool = True

    def _drop_col_si_existe(self, df: pd.DataFrame, col: str) -> pd.DataFrame:
        return df.drop(columns=[col], errors="ignore")

    def _limpiar_parsear_datetime(self, df: pd.DataFrame, col: str) -> pd.DataFrame:
        s = (
            df[col].astype(str)
            .str.replace(r"[\r\n\t]+", " ", regex=True)
            .str.strip()
        )
        s = s.mask(s.eq(""))
        s = s.mask(s.str.lower().eq("nan"))

        dt = pd.to_datetime(s, errors="coerce")
        miss = dt.isna()
        # Segundo intento con formato explícito mm/dd/YYYY HH:MM
        dt.loc[miss] = pd.to_datetime(
            s[miss], format="%m/%d/%Y %H:%M", errors="coerce"
        )

        # Imputación por vecinos: 10 minutos o punto medio
        prev = dt.shift(1)
        nxt = dt.shift(-1)
        mask = dt.isna() & prev.notna() & nxt.notna()

        m10 = mask & ((nxt - prev) == pd.Timedelta(minutes=20))
        dt.loc[m10] = prev.loc[m10] + pd.Timedelta(minutes=10)

        m_mid = mask & dt.isna()
        if m_mid.any():
            mid_ns = (prev[m_mid].astype("int64") + nxt[m_mid].astype("int64")) // 2
            dt.loc[m_mid] = pd.to_datetime(mid_ns)

        df[col] = dt
        return df

    def _imputar_numericos_mediana(self, df: pd.DataFrame) -> pd.DataFrame:
        num_cols = df.select_dtypes(include="number").columns
        medianas = df[num_cols].median()
        df[num_cols] = df[num_cols].fillna(medianas)
        return df

    def _outliers_mediana_rodante(self, df: pd.DataFrame, col_fecha: str) -> pd.DataFrame:
        df = df.sort_values(col_fecha).copy()
        num = df.select_dtypes("number").columns

        Q1, Q3 = df[num].quantile(0.25), df[num].quantile(0.75)
        IQR = Q3 - Q1
        lo, hi = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        mask = (df[num] < lo) | (df[num] > hi)

        for c in num:
            rmed = (
                df[c]
                .rolling(window=self.ventana_mediana, center=True, min_periods=1)
                .median()
            )
            df.loc[mask[c], c] = rmed[mask[c]].fillna(df[c].median())
        return df

    def _features_tiempo(self, df: pd.DataFrame, col_fecha: str) -> pd.DataFrame:
        dt = df[col_fecha]
        df["Day"] = dt.dt.day
        df["Month"] = dt.dt.month
        df["Hour"] = dt.dt.hour
        df["Minute"] = dt.dt.minute
        df["Day of Week"] = dt.dt.dayofweek + 1
        # Quarter
        df["Quarter of Year"] = pd.cut(
            df["Month"],
            bins=[0, 3, 6, 9, 12],
            labels=[1, 2, 3, 4],
            include_lowest=True,
        ).astype(int)
        # Day of Year
        df["Day of Year"] = dt.dt.strftime("%j").astype(int)
        return df

    def _finalizar(self, df: pd.DataFrame, col_fecha: str) -> pd.DataFrame:
        df = df.dropna().copy()
        if self.eliminar_datetime and col_fecha in df.columns:
            df = df.drop(columns=[col_fecha])
        return df

    def ejecutar(self, df_modificado: pd.DataFrame) -> pd.DataFrame:
        df = df_modificado.copy()
        df = self._drop_col_si_existe(df, "mixed_type_col")
        df = self._limpiar_parsear_datetime(df, "DateTime")
        df = self._imputar_numericos_mediana(df)
        df = self._outliers_mediana_rodante(df, "DateTime")
        df = self._features_tiempo(df, "DateTime")
        df = self._finalizar(df, "DateTime")
        return df


# ==========================
# PIPELINE CONJUNTA
# ==========================

def correr_pipeline(
    carpeta_raw: str | Path = "../data/raw",
    carpeta_processed: str | Path = "../data/processed",
    nombre_salida: str = "power_tetouan_city_processed.csv",
    nombre_modificado: str = "power_tetouan_city_modified.csv",
    ventana_mediana: int = 25,
    eliminar_datetime: bool = True,
) -> Path:
    """Ejecuta carga + preprocesamiento y guarda el CSV final."""
    carpeta_processed = Path(carpeta_processed)
    carpeta_processed.mkdir(parents=True, exist_ok=True)

    loader = CargaArchivos(carpeta_raw, nombre_modificado)
    df_modificado = loader.leer()

    pp = Preprocesamiento(ventana_mediana=ventana_mediana, eliminar_datetime=eliminar_datetime)
    df_final = pp.ejecutar(df_modificado)

    ruta_out = carpeta_processed / nombre_salida
    df_final.to_csv(ruta_out, index=False)
    return ruta_out